In [1]:
# Importar librerías necesarias

import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)

In [2]:
# Importar datos

df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)

/tmp/ipykernel_460/2420825280.py:3: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)


# Consigna 1

Columna “reservation_status_date” (CARLA)

1. Limpieza y transformación: 
    -  Esta columna es de tipo object y deberíamos pasarla a tipo date. Para poder hacerlo, vamos a necesitar unificar el formato en el que aparecen las fechas (en algunos casos tienen guiones y en otros barras).
    - Hay 30 filas con fechas posteriores a 2017. En principio no nos preocupa mucho por tratarse de un porcentaje muy pequeño. Intentaremos corregirlo teniendo en cuenta las columnas que tienen información de la fecha de arrival. 

2. Imputación de nulos: 
    - Completar a partir de “arrival _date_year”
    - El resto imputarlas con KNN o iterative.


## Exploración y análisis

In [46]:
# Definir una función para explorar las caraterísticas de una columna rápidamente

def quick_check(df, column_name):
        """ This function is for testing purposes, to quicky check data type and unique values of a column"""
        print (f"\nColumn name: {column_name}")
        print (f"\nData type: {df[column_name].dtype}")
        print (f"\nNot null count: {df[column_name].notnull().sum()}")
        print (f"\nNull count: {df[column_name].isnull().sum()}, {round(df[column_name].isnull().sum()/df.shape[0]*100)}%")
        print (f"\nDuplicated values: {df[column_name].duplicated().sum()}")
        print (f"\nUnique values: \n{df[column_name].unique()}")
        print (f"\nValue counts: \n{df[column_name].value_counts()}")      

In [45]:
# Explorar columna "reservation_status_date"

quick_check(df,"reservation_status_date")


Column name: reservation_status_date

Data type: object

Not null count: 106469

Null count: 76408, 42%

Duplicated values: 181930

Unique values: 
['2015-07-01 00:00:00' '2015-07-02 00:00:00' '2015-07-03 00:00:00'
 '2015-05-06 00:00:00' '2015-04-22 00:00:00' '2015-06-23 00:00:00'
 '2015-07-05 00:00:00' nan '2015-07-07 00:00:00' '2015-07-08 00:00:00'
 '2015-05-11 00:00:00' '2015-07-15 00:00:00' '2015-07-16 00:00:00'
 '2015-05-29 00:00:00' '2015-05-19 00:00:00' '2015-06-19 00:00:00'
 '2015-07-06 00:00:00' '2015-05-23 00:00:00' '2015-05-18 00:00:00'
 '2015-07-09 00:00:00' '2015-06-02 00:00:00' '2015-07-04 00:00:00'
 '2015-06-29 00:00:00' '2015-06-16 00:00:00' '2015-06-18 00:00:00'
 '2015-06-12 00:00:00' '2015-06-09 00:00:00' '2015-05-26 00:00:00'
 '2015-07-11 00:00:00' '2015-07-12 00:00:00' '2015-07-13 00:00:00'
 '2015-07-17 00:00:00' '2015-04-15 00:00:00' '2015-05-13 00:00:00'
 '2015-07-10 00:00:00' '2015-05-20 00:00:00' '2015-05-12 00:00:00'
 '2015-07-14 00:00:00' '2015-06-17 00:00:00

## Conclusiones iniciales (parciales): 
- Algunas fechas tienen formato fecha y hora 'AAAA-MM-DD 00:00:00' y otras solo fecha '2023-04-31'
- En el caso de las que tienen fecha y hora, la hora es siempre '00:00:00'

In [5]:
# Explorar las filas que tienen el formato 'AAAA-MM-DD' ya que parece haber errores en ellas 

# Filtrar dataframe por las filas que coinciden con el formato 'AAAA-MM-DD' en la columna 'reservation_status_date'  
df_filtered = df[df['reservation_status_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]

# Revisar columna 'reservation_status_date' del df filtrado
quick_check(df_filtered, 'reservation_status_date')



Column name: reservation_status_date

Data type: object

Not null count: 45

Null count: 0, 0%

Duplicated values: 0

Unique values: 
['2030-02-30' '2030-11-31' '2029-04-31' '2030-06-31' '2030-09-31'
 '2025-02-30' '2029-11-31' '2023-06-31' '2029-06-31' '2029-02-30'
 '2022-09-31' '2024-09-31' '2028-04-31' '2024-06-31' '2025-11-31'
 '2026-09-31' '2027-09-31' '2023-11-31' '2028-09-31' '2028-02-30'
 '2026-06-31' '2024-04-31' '2023-04-31' '2025-04-31' '2027-04-31'
 '2022-06-31' '2024-02-30' '2025-09-31' '2027-02-30' '2030-04-31'
 '2023-09-31' '2027-06-31' '2026-11-31' '2028-11-31' '2028-06-31'
 '2026-04-31' '2022-11-31' '2025-06-31' '2029-09-31' '2023-02-30'
 '2026-02-30' '2022-04-31' '2022-02-30' '2027-11-31' '2024-11-31']

Value counts: 
2030-02-30    1
2025-04-31    1
2022-06-31    1
2024-02-30    1
2025-09-31    1
2027-02-30    1
2030-04-31    1
2023-09-31    1
2027-06-31    1
2026-11-31    1
2028-11-31    1
2028-06-31    1
2026-04-31    1
2022-11-31    1
2025-06-31    1
2029-09-31    

In [6]:
# Inspeccionar df filtrado 

df_filtered.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
313,Resort Hotel,False,90.0,NaN,July,29.0,12.0,2.0,5.0,2.0,NaN,0.0,FB,NaN,NaN,TA/TO,0.0,NaN,0.0,NaN,A,0.0,5.0,NaN,0.0,NaN,117.22,0.0,1.0,Check-Out,2030-02-30,NaN
7646,Resort Hotel,True,165.0,NaN,August,34.0,18.0,0.0,3.0,2.0,0.0,0.0,HB,NaN,Online TA,TA/TO,0.0,0.0,0.0,D,D,0.0,240.0,NaN,0.0,Transient,222.00,0.0,0.0,Canceled,2030-11-31,NaN
9139,Resort Hotel,True,16.0,NaN,October,NaN,29.0,0.0,1.0,2.0,0.0,0.0,BB,PRT,NaN,Direct,0.0,0.0,0.0,A,A,0.0,NaN,NaN,0.0,Transient,38.00,0.0,1.0,Canceled,2029-04-31,NaN
9493,Resort Hotel,True,183.0,NaN,December,NaN,3.0,2.0,1.0,2.0,NaN,0.0,FB,PRT,Groups,TA/TO,0.0,0.0,0.0,A,A,0.0,38.0,NaN,0.0,NaN,72.00,0.0,0.0,Canceled,2030-06-31,NaN
18350,Resort Hotel,True,82.0,2016.0,2,7.0,7.0,2.0,0.0,2.0,0.0,0.0,BB,NaN,NaN,TA/TO,1.0,1.0,0.0,NaN,A,0.0,67.0,NaN,0.0,NaN,86.00,0.0,0.0,Canceled,2030-09-31,NaN


In [7]:
# Conocer la cantidad de filas que tiene el df filtrado

df_filtered.shape

(45, 32)

## Conclusiones finales

- Todas las fechas con formato 'AAAA-MM-DD' (45) son erróneas ya que contienen días que no existen en el mes, como 31/09 o 30/02 o son fechas de años en el futuro

## Decisión 

- Reemplazar las fechas erróneas, y también las nulas, por una categoría diferente para que no intervengan negativamente en el análisis


## Acciones de limpieza y transformación (NUEVO)

1. Reemplazar las fechas erróneas por nulos 
2. Imputar todos los nulos con KNN
3. Quitar la componente 'HH:MM:SS' (al ser todas 00:00:00 no aportan valor) --> Antes de hacerlo, verificar cómo lo hizo Laura, para que quede alineado con las fechas que trabajó ella. ❗❗ 
4. Cambiar el formato de la columna a "datetime" (usar to_datetime)

In [38]:
# 1) Definir función para reemplazar las fechas erróneas aquellas con formato 'AAAA-MM-DD') por nulos

def reemplazar_fechas_erroneas(fecha):
    # Verificar si el valor es una cadena de texto
    if isinstance(fecha, str):
        # Expresión regular para identificar fechas en formato 'AAAA-MM-DD'
        patron = r'^\d{4}-\d{2}-\d{2}$'
        if re.match(patron, fecha):
            return np.nan
    # Si no es una cadena de texto o no coincide con el patrón, devolver el valor original
    return fecha


In [39]:
# Defino los parámetros para llamar a la función

column = 'reservation_status_date'
df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)

/tmp/ipykernel_460/450431777.py:4: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)


In [41]:
quick_check(df,column)


Column name: reservation_status_date

Data type: object

Not null count: 106514

Null count: 76363, 42%

Duplicated values: 181885

Unique values: 
['2015-07-01 00:00:00' '2015-07-02 00:00:00' '2015-07-03 00:00:00'
 '2015-05-06 00:00:00' '2015-04-22 00:00:00' '2015-06-23 00:00:00'
 '2015-07-05 00:00:00' nan '2015-07-07 00:00:00' '2015-07-08 00:00:00'
 '2015-05-11 00:00:00' '2015-07-15 00:00:00' '2015-07-16 00:00:00'
 '2015-05-29 00:00:00' '2015-05-19 00:00:00' '2015-06-19 00:00:00'
 '2015-07-06 00:00:00' '2015-05-23 00:00:00' '2015-05-18 00:00:00'
 '2015-07-09 00:00:00' '2015-06-02 00:00:00' '2015-07-04 00:00:00'
 '2015-06-29 00:00:00' '2015-06-16 00:00:00' '2015-06-18 00:00:00'
 '2015-06-12 00:00:00' '2015-06-09 00:00:00' '2015-05-26 00:00:00'
 '2015-07-11 00:00:00' '2015-07-12 00:00:00' '2015-07-13 00:00:00'
 '2015-07-17 00:00:00' '2015-04-15 00:00:00' '2015-05-13 00:00:00'
 '2015-07-10 00:00:00' '2015-05-20 00:00:00' '2015-05-12 00:00:00'
 '2015-07-14 00:00:00' '2015-06-17 00:00:00

In [42]:
# Llamo a la función

df[column] = df[column].apply(reemplazar_fechas_erroneas)

In [43]:
quick_check(df,column)


Column name: reservation_status_date

Data type: object

Not null count: 106469

Null count: 76408, 42%

Duplicated values: 181930

Unique values: 
['2015-07-01 00:00:00' '2015-07-02 00:00:00' '2015-07-03 00:00:00'
 '2015-05-06 00:00:00' '2015-04-22 00:00:00' '2015-06-23 00:00:00'
 '2015-07-05 00:00:00' nan '2015-07-07 00:00:00' '2015-07-08 00:00:00'
 '2015-05-11 00:00:00' '2015-07-15 00:00:00' '2015-07-16 00:00:00'
 '2015-05-29 00:00:00' '2015-05-19 00:00:00' '2015-06-19 00:00:00'
 '2015-07-06 00:00:00' '2015-05-23 00:00:00' '2015-05-18 00:00:00'
 '2015-07-09 00:00:00' '2015-06-02 00:00:00' '2015-07-04 00:00:00'
 '2015-06-29 00:00:00' '2015-06-16 00:00:00' '2015-06-18 00:00:00'
 '2015-06-12 00:00:00' '2015-06-09 00:00:00' '2015-05-26 00:00:00'
 '2015-07-11 00:00:00' '2015-07-12 00:00:00' '2015-07-13 00:00:00'
 '2015-07-17 00:00:00' '2015-04-15 00:00:00' '2015-05-13 00:00:00'
 '2015-07-10 00:00:00' '2015-05-20 00:00:00' '2015-05-12 00:00:00'
 '2015-07-14 00:00:00' '2015-06-17 00:00:00

In [ ]:
# 2) Imputar nulos con KNN (usando la función de Laura)

In [ ]:
# 3) 3. Quitar la componente 'HH:MM:SS' (al ser todas 00:00:00 no aportan valor) 
# ❗❗ Antes de hacerlo, verificar cómo lo hizo Laura, para que quede alineado con las fechas que trabajó ella.  

# Función para transformar fechas con formato 'AAAA-MM-DD 00:00:00' en 'AAAA-MM-DD'
def quitar_horas (fecha):
    """ 
    """
    # Verificar si el valor es una cadena de texto
    if isinstance(fecha, str):
        fecha = fecha.split()[0]
        return fecha
    # Si no es una cadena de texto, devolver el valor original
    return fecha

In [ ]:
# 4) Cambiar el formato a date time

df[column] = pd.to_datetime(df[column])

## Acciones de limpieza y transformación (DESCARTADO)

1. Reemplazar las fechas erróneas con una categoría especial "1677-09-21" (es la fecha mínima que existe en Pandas) 
2. Reemplazar los nulos también con "1677-09-21"
3. Quitar la componente 'HH:MM:SS' (al ser todas 00:00:00 no aportan valor)
4. Cambiar el formato de la columna a "datetime" (usar to_datetime)


In [26]:
df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)

/tmp/ipykernel_460/1059111946.py:1: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)


In [9]:
# Definición de funciones de transformación

# Función para reemplazar las fechas erróneas con "0000-00-00"
def reemplazar_fechas_erroneas(fecha):
    # Verificar si el valor es una cadena de texto
    if isinstance(fecha, str):
        # Expresión regular para identificar fechas en formato 'AAAA-MM-DD'
        patron = r'^\d{4}-\d{2}-\d{2}$'
        if re.match(patron, fecha):
            return '1677-09-22'
    # Si no es una cadena de texto o no coincide con el patrón, devolver el valor original
    return fecha

# Función para transformar fechas con formato 'AAAA-MM-DD 00:00:00' en 'AAAA-MM-DD'
def quitar_horas (fecha):
    # Verificar si el valor es una cadena de texto
    if isinstance(fecha, str):
        fecha = fecha.split()[0]
        return fecha
    # Si no es una cadena de texto, devolver el valor original
    return fecha

In [10]:
# Aplicar las transformación a la columna

column = 'reservation_status_date'
df[column] = df[column].apply(reemplazar_fechas_erroneas)
df[column] = df[column].apply(quitar_horas)
df[column] = df[column].fillna('1677-09-22')
df[column] = pd.to_datetime(df[column])


In [16]:
print (type(df))

<class 'pandas.core.frame.DataFrame'>


In [11]:
# Print for testing 

quick_check(df, column)


Column name: reservation_status_date

Data type: datetime64[ns]

Not null count: 182877

Null count: 0, 0%

Duplicated values: 181930

Unique values: 
['2015-07-01T00:00:00.000000000' '2015-07-02T00:00:00.000000000'
 '2015-07-03T00:00:00.000000000' '2015-05-06T00:00:00.000000000'
 '2015-04-22T00:00:00.000000000' '2015-06-23T00:00:00.000000000'
 '2015-07-05T00:00:00.000000000' '1677-09-22T00:00:00.000000000'
 '2015-07-07T00:00:00.000000000' '2015-07-08T00:00:00.000000000'
 '2015-05-11T00:00:00.000000000' '2015-07-15T00:00:00.000000000'
 '2015-07-16T00:00:00.000000000' '2015-05-29T00:00:00.000000000'
 '2015-05-19T00:00:00.000000000' '2015-06-19T00:00:00.000000000'
 '2015-07-06T00:00:00.000000000' '2015-05-23T00:00:00.000000000'
 '2015-05-18T00:00:00.000000000' '2015-07-09T00:00:00.000000000'
 '2015-06-02T00:00:00.000000000' '2015-07-04T00:00:00.000000000'
 '2015-06-29T00:00:00.000000000' '2015-06-16T00:00:00.000000000'
 '2015-06-18T00:00:00.000000000' '2015-06-12T00:00:00.000000000'
 '2

## Imputación de nulos en la columna "reservation_status_date"

### Imputar 'reservation_status_date' a partir de la fecha de arrival

Para poder hacer esta imputación, es necesario conocer el 'reservation_status' asociado:

- Si el reservation status es "Check-out", entonces la "reservation_status_date" es igual a "arrival_date" + "total_stays".
- Si el reservation status es "No-Show", entonces la "reservation_status_date" a "arrival_date".
- Si el reservation status es "Canceled", no es posible deducir la "reservation_status_date".

In [12]:
# ¿Cuántos registros con reservation status "Check-out" tienen "reservation_status_date" nulo?

df[(df['reservation_status'] == 'Check-out') & (df['reservation_status_date'] == '1677-09-22T00:00:00.000000000')].shape[0]


0

In [13]:
# ¿Cuántos registros con reservation status "No-show" tienen "reservation_status_date" nulo?

df[(df['reservation_status'] == 'No-show') & (df['reservation_status_date'] == '1677-09-22T00:00:00.000000000')].shape[0]

0

In [14]:
# ¿Cuántos registros con reservation status "Canceled" tienen "reservation_status_date" nulo?

df[(df['reservation_status'] == 'Canceled') & (df['reservation_status_date'] == '1677-09-22T00:00:00.000000000')].shape[0]

4682

### CONCLUSIÓN: No es posible imputar la 'reservation_status_date' a partir de la fecha de arrival ❌❌❌

Esto se debe a que los registros que tienen 'reservation_status_date' tienen el 'reservation_status' "Canceled" o Null.

## Acciones de imputación de nulos

- Imputar con KNN o iterative.

# Consigna 2 - Imputación de nulos por "Unknown"

- Definir una función para imputar valores nulos por una categoría llamada "Unknown".
- Esta función se aplicará en las columnas "Country", "Company" y "Agent".

In [27]:
# 1) Defino la función para imputar nulos por 'Unknown'

def imputar_por_unknown(dataframe, lista_columnas):
        """
        Esta función itera sobre una lista de columnas dentro de un dataframe y reemplaza los nulos por 'Unknown'.

        Args:
            dataframe (pandas dataframe): Es el dataframe que contiene los datos.
            lista_columnas (lista): Lista de columnas existentes en el dataframe sobre las cuales se quiere imputar los nulos.

        Returns:
            dataframe: Dataframe actualizado.
        """
        for columna in lista_columnas:
            # Verificar si la columna existe en el dataframe
            if columna in dataframe:
                # Si existe, reemplazar nulos por 'Unknown'
                dataframe[columna] = dataframe[columna].fillna("Unknown")
            else:
                # Si no existe, imprimir un mensaje de error
                print(f"❌ La columna '{columna}' no existe en el DataFrame.")

        return dataframe

In [28]:
# 2) Defino los argumentos para llamar la función: 

# Creo la lista de columnas sobre las cuales quiero aplicar la función 
lista_columnas = ['country', 'company', 'agent']

# Me traigo el dataframe sin modificar
df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)

/tmp/ipykernel_460/2985356423.py:5: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)


In [29]:
# 3) Chequeo previo: Reviso los nulos de las columnas antes de aplicarles la función

quick_check(df, 'country')

quick_check(df, 'company')

quick_check(df, 'agent')


Column name: country

Data type: object

Not null count: 65054

Null count: 117823, 64%

Duplicated values: 182713

Unique values: 
['PRT' nan 'GBR' 'USA' 'ESP' 'IRL' 'FRA' 'ROU' 'NOR' 'OMN' 'ARG' 'DEU'
 'CHE' 'GRC' 'NLD' 'DNK' 'RUS' 'POL' 'AUS' 'EST' 'CZE' 'BRA' 'BEL' 'CN'
 'SWE' 'FIN' 'MOZ' 'SVN' 'MAR' 'ITA' 'UKR' 'SMR' 'LVA' 'PRI' 'CHL' 'CHN'
 'LTU' 'LUX' 'AUT' 'TUR' 'MEX' 'ZAF' 'AGO' 'ISR' 'IND' 'CYM' 'ZMB' 'CPV'
 'ZWE' 'DZA' 'KOR' 'HUN' 'ARE' 'TUN' 'JAM' 'ALB' 'HRV' 'HKG' 'AND' 'GIB'
 'URY' 'BLR' 'JEY' 'CYP' 'MDV' 'FJI' 'KAZ' 'PAK' 'IDN' 'LBN' 'PHL' 'COL'
 'SEN' 'GEO' 'AZE' 'BHR' 'NZL' 'THA' 'DOM' 'MYS' 'VEN' 'ARM' 'LKA' 'CUB'
 'CMR' 'IRN' 'BIH' 'NGA' 'COM' 'BGR' 'CIV' 'SRB' 'JOR' 'SYR' 'BDI' 'SGP'
 'KWT' 'PLW' 'QAT' 'SVK' 'SUR' 'MLT' 'MWI' 'MDG' 'ISL' 'JPN' 'CAF' 'TGO'
 'TWN' 'DJI' 'VNM' 'PER' 'EGY' 'SAU' 'KNA' 'ETH' 'ECU' 'IRQ' 'KHM' 'MCO'
 'BGD' 'TJK' 'NIC' 'GGY' 'BEN' 'VGB' 'CRI' 'TZA' 'GAB' 'MKD' 'TMP' 'GLP'
 'LIE' 'GNB' 'MAC' 'IMN' 'UMI' 'MYT' 'GHA' 'FRO' 'MMR' 'PAN' 'MUS' 

In [30]:
# 4) Llamo a la función
imputar_por_unknown (df, lista_columnas)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.000000,2015.0,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,Unknown,Unknown,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,False,737.000000,NaN,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,Unknown,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,Unknown,Unknown,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,False,7.000000,2015.0,July,27.0,1.0,0.000000,1.000000,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,Unknown,Unknown,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,Resort Hotel,False,13.000000,NaN,July,27.0,1.0,0.000000,1.000000,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,Unknown,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,Resort Hotel,False,14.000000,NaN,July,NaN,1.0,0.000000,2.000000,2.0,NaN,0.0,BB,Unknown,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,Unknown,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182872,NaN,NaN,224.337762,NaN,NaN,NaN,NaN,19.005545,19.819823,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182873,NaN,NaN,390.141963,NaN,NaN,NaN,NaN,14.751794,19.989726,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182874,NaN,NaN,230.689826,NaN,NaN,NaN,NaN,11.409496,20.461372,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182875,NaN,NaN,304.888534,NaN,NaN,NaN,NaN,16.744472,15.400773,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# 5) Chequeo porterior: Reviso los nulos de las columnas luego de aplicarles la función para verificar que los nulos hayan sido imputados 

quick_check(df, 'country')

quick_check(df, 'company')

quick_check(df, 'agent')


Column name: country

Data type: object

Not null count: 182877

Null count: 0, 0%

Duplicated values: 182713

Unique values: 
['PRT' 'Unknown' 'GBR' 'USA' 'ESP' 'IRL' 'FRA' 'ROU' 'NOR' 'OMN' 'ARG'
 'DEU' 'CHE' 'GRC' 'NLD' 'DNK' 'RUS' 'POL' 'AUS' 'EST' 'CZE' 'BRA' 'BEL'
 'CN' 'SWE' 'FIN' 'MOZ' 'SVN' 'MAR' 'ITA' 'UKR' 'SMR' 'LVA' 'PRI' 'CHL'
 'CHN' 'LTU' 'LUX' 'AUT' 'TUR' 'MEX' 'ZAF' 'AGO' 'ISR' 'IND' 'CYM' 'ZMB'
 'CPV' 'ZWE' 'DZA' 'KOR' 'HUN' 'ARE' 'TUN' 'JAM' 'ALB' 'HRV' 'HKG' 'AND'
 'GIB' 'URY' 'BLR' 'JEY' 'CYP' 'MDV' 'FJI' 'KAZ' 'PAK' 'IDN' 'LBN' 'PHL'
 'COL' 'SEN' 'GEO' 'AZE' 'BHR' 'NZL' 'THA' 'DOM' 'MYS' 'VEN' 'ARM' 'LKA'
 'CUB' 'CMR' 'IRN' 'BIH' 'NGA' 'COM' 'BGR' 'CIV' 'SRB' 'JOR' 'SYR' 'BDI'
 'SGP' 'KWT' 'PLW' 'QAT' 'SVK' 'SUR' 'MLT' 'MWI' 'MDG' 'ISL' 'JPN' 'CAF'
 'TGO' 'TWN' 'DJI' 'VNM' 'PER' 'EGY' 'SAU' 'KNA' 'ETH' 'ECU' 'IRQ' 'KHM'
 'MCO' 'BGD' 'TJK' 'NIC' 'GGY' 'BEN' 'VGB' 'CRI' 'TZA' 'GAB' 'MKD' 'TMP'
 'GLP' 'LIE' 'GNB' 'MAC' 'IMN' 'UMI' 'MYT' 'GHA' 'FRO' 'MMR' 'PAN' 'MUS'

# Consigna 3 - Imputación de nulos por la moda

- Definir una función para imputar valores nulos por la moda.
- Esta función se aplicará en las columnas "is_repeated_guest" y "customer_type".


In [33]:
# 1) Defino la función para imputar nulos por la moda

def imputar_por_moda (dataframe, lista_columnas):
        """
        Esta función itera sobre una lista de columnas dentro de un dataframe y reemplaza los nulos por la moda.

        Args:
            dataframe (pandas dataframe): Es el dataframe que contiene los datos.
            lista_columnas (lista): Lista de columnas existentes en el dataframe sobre las cuales se quiere imputar los nulos.

        Returns:
            dataframe: Dataframe actualizado.
        """
        for columna in lista_columnas:
            # Verificar si la columna existe en el dataframe
            if columna in dataframe.columns:
                # Si existe, calcular la moda y reemplazar los nulos con la moda
                moda = dataframe[columna].mode()[0]
                dataframe[columna] = dataframe[columna].fillna(moda)
            else:
                print(f"❌ La columna '{columna}' no existe en el DataFrame.")
        return dataframe

In [34]:
# 2) creo la lista de columnas sobre las cuales quiero aplicar la función
lista_columnas = ["is_repeated_guest","customer_type"]


In [35]:
# 3) Chequeo previo: Reviso los nulos de las columnas antes de aplicarles la función

quick_check(df, "is_repeated_guest")

quick_check(df, "customer_type")


Column name: is_repeated_guest

Data type: float64

Not null count: 114376

Null count: 68501, 37%

Duplicated values: 182874

Unique values: 
[ 0. nan  1.]

Value counts: 
0.0    110730
1.0      3646
Name: is_repeated_guest, dtype: int64

Column name: customer_type

Data type: object

Not null count: 94199

Null count: 88678, 48%

Duplicated values: 182872

Unique values: 
['Transient' nan 'Contract' 'Transient-Party' 'Group']

Value counts: 
Transient          70610
Transient-Party    19839
Contract            3290
Group                460
Name: customer_type, dtype: int64


In [36]:
# 4) Llamo a la función
imputar_por_moda (df, lista_columnas)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.000000,2015.0,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,Unknown,Unknown,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,False,737.000000,NaN,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,Unknown,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,Unknown,Unknown,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,False,7.000000,2015.0,July,27.0,1.0,0.000000,1.000000,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,Unknown,Unknown,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,Resort Hotel,False,13.000000,NaN,July,27.0,1.0,0.000000,1.000000,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,Unknown,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,Resort Hotel,False,14.000000,NaN,July,NaN,1.0,0.000000,2.000000,2.0,NaN,0.0,BB,Unknown,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,Unknown,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182872,NaN,NaN,224.337762,NaN,NaN,NaN,NaN,19.005545,19.819823,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,Transient,NaN,NaN,NaN,NaN,NaN,NaN
182873,NaN,NaN,390.141963,NaN,NaN,NaN,NaN,14.751794,19.989726,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,Transient,NaN,NaN,NaN,NaN,NaN,NaN
182874,NaN,NaN,230.689826,NaN,NaN,NaN,NaN,11.409496,20.461372,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,Transient,NaN,NaN,NaN,NaN,NaN,NaN
182875,NaN,NaN,304.888534,NaN,NaN,NaN,NaN,16.744472,15.400773,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,Transient,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# 5) Chequeo porterior: Reviso los nulos de las columnas luego de aplicarles la función para verificar que los nulos hayan sido imputados 

quick_check(df, "is_repeated_guest")

quick_check(df, "customer_type")


Column name: is_repeated_guest

Data type: float64

Not null count: 182877

Null count: 0, 0%

Duplicated values: 182875

Unique values: 
[0. 1.]

Value counts: 
0.0    179231
1.0      3646
Name: is_repeated_guest, dtype: int64

Column name: customer_type

Data type: object

Not null count: 182877

Null count: 0, 0%

Duplicated values: 182873

Unique values: 
['Transient' 'Contract' 'Transient-Party' 'Group']

Value counts: 
Transient          159288
Transient-Party     19839
Contract             3290
Group                 460
Name: customer_type, dtype: int64
